# 梯度检验

假设你们公司有一个全球支付项目，老板要求你构建一个AI模型来判断支付是否可靠。

由于反向传播的实现是有一定难度的，所以有时候会出现bug。

因为支付项目对准确性要求很高，所以你的老板要求你百分之百保证你的反向传播是没有问题的。

你老板说："你要证明给我看，证明你的反向传播是完全正确的"。

那么，这时，最好的证明手段就是我们前面文章中提到的梯度检验。

In [1]:
import numpy as np
from testCases import *
from gc_utils import sigmoid, relu, dictionary_to_vector, vector_to_dictionary, gradients_to_vector

## 1) 梯度检验的原理是什么？

我们知道，反向传播可以计算出梯度。假设我们要计算$\theta$相当于成本函数$J$的梯度$\frac{\partial J}{\partial \theta}$。

因为前向传播是比较容易实现的，你也非常自信地确保你的前向传播是正确的，也就是说你百分之百地确信你的成本$J$是正确的。

所以，你就可以用计算成本$J$的前向传播代码来验证计算梯度$\frac{\partial J}{\partial \theta}$的反向传播代码。

我们回顾一下数学中导数（梯度）的定义：
$$ \frac{\partial J}{\partial \theta} = \lim_{\varepsilon \to 0} \frac{J(\theta + \varepsilon) - J(\theta - \varepsilon)}{2 \varepsilon} \tag{1}$$

我们来捋一捋：
- $\frac{\partial J}{\partial \theta}$ 是你通过反向传播计算得到的，你需要验证它是否准确；
- 我们可以用另外一种方式计算出$\frac{\partial J}{\partial \theta}$，如果它与反向传播计算得到的一样，那么反向传播就是正确的；
- 这另外的计算方式就是我们上面的公式，我们可以用前向传播分别计算出$J(\theta + \varepsilon)$和$J(\theta - \varepsilon)$来求得$\frac{\partial J}{\partial \theta}$。

## 2) 1维的梯度检验

假设我们有一个简单的1维线性函数$J(\theta) = \theta x$。这个函数（模型）只有一个参数$\theta$，$x$是输入。

下面我们会用代码来计算出$J(\theta)$（用前向传播计算出成本），然后计算出$\frac{\partial J}{\partial \theta}$（用反向传播计算出梯度）。

最后我们用梯度检验来证明反向传播计算出来的梯度是正确的。

<img src="images/1Dgrad_kiank.png" style="width:600px;height:250px;">
<caption><center>图 1：1维线性模型</center></caption>

上面的流程图显示出了关键的步骤：输入 $x$；然后计算出 $J(\theta)$（前向传播）；然后计算出梯度 $\frac{\partial J}{\partial \theta}$（反向传播）。

In [2]:
# 前向传播
def forward_propagation(x, theta):
    J = np.dot(theta, x)
    return J


In [3]:
x, theta = 2, 4
J = forward_propagation(x, theta)
print ("J = " + str(J))

J = 8


In [4]:
# 反向传播
def backward_propagation(x, theta):
    # 这个函数的导数就是x
    dtheta = x
    return dtheta


In [5]:
x, theta = 2, 4
dtheta = backward_propagation(x, theta)
print ("dtheta = " + str(dtheta))

dtheta = 2


下面我们将用梯度检验来确认上面反向传播计算出来的梯度dtheta是正确的。

**主要步骤如下**：
- 我们先通过下面5小步来利用前向传播计算出梯度，这个梯度我们用gradapprox表示。
    1. $\theta^{+} = \theta + \varepsilon$
    2. $\theta^{-} = \theta - \varepsilon$
    3. $J^{+} = J(\theta^{+})$
    4. $J^{-} = J(\theta^{-})$
    5. $gradapprox = \frac{J^{+} - J^{-}}{2  \varepsilon}$
- 然后利用上面的反向传播也计算出一个梯度，这个梯度我们用grad表示。
- 最后，我们用下面的公式来计算gradapprox和grad这两个梯度相差多远。
$$ difference = \frac {\mid\mid grad - gradapprox \mid\mid_2}{\mid\mid grad \mid\mid_2 + \mid\mid gradapprox \mid\mid_2} \tag{2}$$   
- 如果两个梯度相差小于$10^{-7}$，那么说明这两个梯度很接近，也就是说，你的反向传播是正确的；否则，说明你的反向传播里面有问题。

In [6]:
def gradient_check(x, theta, epsilon=1e-7):
    # 利用前向传播计算出一个梯度
    thetaplus = theta + epsilon
    thetaminus = theta - epsilon
    J_plus = forward_propagation(x, thetaplus)
    J_minus = forward_propagation(x, thetaminus)
    gradapprox = (J_plus - J_minus) / (2 * epsilon)

    # 利用反向传播也计算出一个梯度
    grad = backward_propagation(x, theta)

    # 对比两个梯度相差多远
    numerator = np.linalg.norm(grad - gradapprox)
    denominator = np.linalg.norm(grad) + np.linalg.norm(gradapprox)
    difference = numerator / denominator

    if difference < 1e-7:
        print("反向传播是正确的!")
    else:
        print("反向传播有问题！")

    return difference


In [7]:
x, theta = 2, 4
difference = gradient_check(x, theta)
print("difference = " + str(difference))

反向传播是正确的!
difference = 2.919335883291695e-10


两个梯度相差小于$10^{-7}$，说明上面的反向传播得到的梯度是正确的。

但是通常情况下，神经网络的成本函数不仅仅只有一个1维的参数，$\theta$通常是由多个$W^{[l]}$和$b^{[l]}$矩阵构成的。

所以学会如何给多维参数做梯度检验是很重要的，下面我们就来学习多维参数的梯度检验！

## 3) 多维梯度检验

下图展示了支付可靠度预测模型的前向传播和反向传播流程。

<img src="images/NDgrad_kiank.png" style="width:600px;height:400px;">
<caption><center>图 2：神经网络流程图<br>LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SIGMOID</center></caption>

下面就是前向传播和反向传播的代码实现。

In [8]:
def forward_propagation_n(X, Y, parameters):
    m = X.shape[1]
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    W3 = parameters["W3"]
    b3 = parameters["b3"]

    # LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SIGMOID
    Z1 = np.dot(W1, X) + b1
    A1 = relu(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = relu(Z2)
    Z3 = np.dot(W3, A2) + b3
    A3 = sigmoid(Z3)

    logprobs = np.multiply(-np.log(A3), Y) + np.multiply(-np.log(1 - A3), 1 - Y)
    cost = 1. / m * np.sum(logprobs)
    
    cache = (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3)
    
    return cost, cache

In [14]:
def backward_propagation_n(X, Y, cache):    
    m = X.shape[1]
    (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3) = cache
    
    dZ3 = A3 - Y
    dW3 = 1. / m * np.dot(dZ3, A2.T)
    db3 = 1. / m * np.sum(dZ3, axis=1, keepdims=True)
    
    dA2 = np.dot(W3.T, dZ3)
    dZ2 = np.multiply(dA2, np.int64(A2 > 0))
    dW2 = 1. / m * np.dot(dZ2, A1.T)  # ~~
    db2 = 1. / m * np.sum(dZ2, axis=1, keepdims=True)
    
    dA1 = np.dot(W2.T, dZ2)
    dZ1 = np.multiply(dA1, np.int64(A1 > 0))
    dW1 = 1. / m * np.dot(dZ1, X.T)
    db1 = 1. / m * np.sum(dZ1, axis=1, keepdims=True) # ~~
    
    gradients = {"dZ3": dZ3, "dW3": dW3, "db3": db3,
                 "dA2": dA2, "dZ2": dZ2, "dW2": dW2, "db2": db2,
                 "dA1": dA1, "dZ1": dZ1, "dW1": dW1, "db1": db1}
    
    return gradients

上面反向传播的实现是正确吗？下面我们将用梯度检验来验证它。

**如何进行多维梯度检验**：

像1维检验时一样，我们还是使用下面的这个公式：

$$ \frac{\partial J}{\partial \theta} = \lim_{\varepsilon \to 0} \frac{J(\theta + \varepsilon) - J(\theta - \varepsilon)}{2 \varepsilon} \tag{1}$$

但是，多维检验中的$\theta$不再是一个数值了，而是一个字典了，字典里面包含了很多个参数。我给大家实现了一个函数`dictionary_to_vector()`，用它可以将这个字典转换成一个向量，它会改变字典里参数(W1, b1, W2, b2, W3, b3)的维度并且将它们连接起来构成一个大向量，这个向量我们用"values"来表示。

同时也给大家提供了另外一个逆操作的函数`vector_to_dictionary`，它会将向量转换回字典形式。

<img src="images/dictionary_to_vector.png" style="width:600px;height:400px;">
<caption><center>图 2：dictionary_to_vector() 和 vector_to_dictionary()<br>在后面的gradient_check_n()函数中你将使用它们来进行梯度验证</center></caption>

当然，我们也会将梯度gradients对应的字典也转换成一个大向量，转换函数是`gradients_to_vector()`。这些函数我都为大家写好了，大家不用担心，把精力放在重点要理解的理论上就行了。

为了便于大家理解下面的梯度检验函数gradient_check_n()，我先给出它的伪码实现，这有助于大家理解它的逻辑。

For each i in num_parameters:
- To compute `J_plus[i]`：
    1. Set $\theta^{+}$ to `np.copy(parameters_values)`
    2. Set $\theta^{+}_i$ to $\theta^{+}_i + \varepsilon$
    3. Calculate $J^{+}_i$ using to `forward_propagation_n(x, y, vector_to_dictionary(`$\theta^{+}$ `))`
- To compute `J_minus[i]`：do the same thing with $\theta^{-}$
- Compute $gradapprox[i] = \frac{J^{+}_i - J^{-}_i}{2 \varepsilon}$

上面的循环完了后，你就通过前向传播的方式获得了梯度gradapprox，它的每一个元素gradapprox[i]就对应着每一个参数`parameter_values[i]`相关的梯度。然后你再用1维梯度检验时相同的方式来对比梯度gradapprox和反向传播得到的梯度相差远不远。对比公式还是下面这个：
$$ difference = \frac {\| grad - gradapprox \|_2}{\| grad \|_2 + \| gradapprox \|_2 } \tag{3}$$

In [25]:
def gradient_check_n(parameters, gradients, X, Y, epsilon=1e-7):
    # 将参数字典转化为列向量
    parameters_values, _ = dictionary_to_vector(parameters)
    # 将反向传播计算的梯度字典转化为列向量
    grad = gradients_to_vector(gradients)

    # 创建存储结果的容器（列向量）
    num_parameters = parameters_values.shape[0]
    J_plus = np.zeros((num_parameters, 1))
    J_minus = np.zeros((num_parameters, 1))
    gradapprox = np.zeros((num_parameters, 1))

    # 遍历每一个参数，并计算出该参数的gradapprox
    # 每个参数都有一个对应的gradapprox
    # gradapprox是该参数对于成本函数的估计梯度
    for i in range(num_parameters):
        # 给参数加上epsilon
        thetaplus =  np.copy(parameters_values)
        thetaplus[i][0] = thetaplus[i][0] + epsilon
        # 利用前向传播计算出该参数加上epsilon后的成本
        J_plus[i], _ =  forward_propagation_n(X, Y, vector_to_dictionary(thetaplus))

        # 给参数减去epsilon
        thetaminus = np.copy(parameters_values)
        thetaminus[i][0] = thetaminus[i][0] - epsilon
        # 利用前向传播计算出该参数减去epsilon后的成本
        J_minus[i], _ = forward_propagation_n(X, Y, vector_to_dictionary(thetaminus))

        # 由上面计算得到的两个成本，计算出该参数的gradapprox
        gradapprox[i] = (J_plus[i] - J_minus[i]) / (2 * epsilon)

    # 求出gradapprox和grad的差距
    numerator = np.linalg.norm(grad - gradapprox)
    denominator = np.linalg.norm(grad) + np.linalg.norm(gradapprox)
    difference = numerator / denominator

    if difference > 2e-7:
        print("\033[93m" + "反向传播有问题! difference = " + str(difference) + "\033[0m")
    else:
        print("\033[92m" + "反向传播很完美! difference = " + str(difference) + "\033[0m")

    return difference

In [26]:
X, Y, parameters = gradient_check_n_test_case()
cost, cache = forward_propagation_n(X, Y, parameters)
gradients = backward_propagation_n(X, Y, cache)
difference = gradient_check_n(parameters, gradients, X, Y)

反向传播很完美! difference = 1.1890913023330276e-07


**注意**：

- 梯度检验是很缓慢的。通过$\frac{\partial J}{\partial \theta} \approx  \frac{J(\theta + \varepsilon) - J(\theta - \varepsilon)}{2 \varepsilon}$来计算梯度非常消耗计算力。所以，我们不会在训练的每一个回合都执行梯度检验，仅仅偶尔执行几次。
- 梯度检验是无法与dropout共存的，所以在执行梯度检验时，要把dropout关掉，检验完毕后再开启。

<font color='blue'>
本次实战编程需要记住的几点：

- 梯度检验通过用前向传播的方式求出一个梯度，然后将其与反向传播求出的梯度进行对比来判断梯度是否正确。

- 梯度检验很浪费计算力，所以只在需要验证代码是否正确时才开启，确认代码没有问题后，就关闭掉梯度检验。 